# Reinforcement Learning

- Jorge Caballeros Pérez - 20009
- Mario de León - 19019
- Pablo Escobar - 20936 

20/8/2024


### Responda a cada de las siguientes preguntas de forma clara y lo más completamente posible.
#### **1. ¿Qué es Prioritized sweeping para ambientes determinísticos?**
Es un método en el que se priorizan actualizaciones en estados que más impactan la política en ambientes donde las acciones siempre producen el mismo resultado, lo que hace que el aprendizaje sea más rápido y eficiente.

#### **2. ¿Qué es Trajectory Sampling?**
Es un enfoque que toma muestras de trayectorias completas (secuencia de estados y acciones) para estimar el valor de una política o entrenar un modelo, en lugar de actualizar estado por estado.

#### **3. ¿Qué es Upper Confidence Bounds para Árboles (UCT por sus siglas en inglés)?**
Es un algoritmo que equilibra exploración y explotación en árboles de decisión, usando una fórmula que agrega un término de confianza para explorar ramas menos conocidas mientras sigue explotando las mejores opciones conocidas.

### Task 2

In [56]:
import gymnasium as gym
import numpy as np

# Crear el entorno FrozenLake-v1
env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=True)

# Definir una función para simular la interacción con el entorno
def simulate_action(env, state, action):
    env.reset()
    env.env.state = state
    next_state, reward, done, _, _ = env.step(action)
    return next_state, reward, done

# Variables del entorno
state_space = env.observation_space.n
action_space = env.action_space.n


In [57]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = {}
        self.visits = 0
        self.value = 0.0
    
    def expand(self, action, next_state):
        if action not in self.children:
            self.children[action] = Node(next_state, parent=self)
        return self.children[action]
    
    def update(self, reward):
        self.visits += 1
        self.value += reward
        if self.parent:
            self.parent.update(reward)

    def is_fully_expanded(self):
        return len(self.children) == action_space


In [58]:
def ucb1(node, c_param=1.4):
    # Calcula el valor UCB1 para un nodo
    return node.value / (node.visits + 1e-6) + c_param * np.sqrt(np.log(node.parent.visits + 1) / (node.visits + 1e-6))

def select_best_action(node, c_param=1.4):
    return max(node.children.items(), key=lambda item: ucb1(item[1], c_param))[0]



In [59]:
def simulate(env, node, max_depth=100):
    current_node = node
    total_reward = 0
    depth = 0
    
    while depth < max_depth:
        action = np.random.choice(action_space)
        next_state, reward, done = simulate_action(env, current_node.state, action)
        
        if action not in current_node.children:
            current_node = current_node.expand(action, next_state)
        
        total_reward += reward
        depth += 1
        
        if done:
            break
    
    return total_reward


In [60]:
def expand_node(node):
    action = np.random.choice(action_space)
    next_state, reward, done = simulate_action(env, node.state, action)
    
    if action not in node.children:
        return node.expand(action, next_state), reward, done
    else:
        return node, 0, False

def backpropagate(node, reward):
    while node is not None:
        node.visits += 1
        node.value += reward
        node = node.parent


In [61]:
def mcts(env, root, n_simulations=1000):
    for _ in range(n_simulations):
        node = root
        
        # Fase de selección
        while node.is_fully_expanded() and node.children:
            action = select_best_action(node)
            node = node.children[action]
        
        # Fase de expansión
        node, reward, done = expand_node(node)
        
        # Fase de simulación solo si no es un estado terminal
        if not done:
            total_reward = simulate(env, node)
            total_reward += reward  # Suma la recompensa inmediata
        else:
            total_reward = reward  # Si es terminal, solo usa la recompensa inmediata
        
        # Fase de retropropagación
        backpropagate(node, total_reward)
    
    return select_best_action(root)


# Crear la raíz del árbol en el estado inicial
initial_state = env.reset()[0]
root = Node(initial_state)

# Ejecutar MCTS
best_action = mcts(env, root)
actions = ['Izquierda','Abajo','Derecha','Arriba'] 


print(f"La mejor acción seleccionada es: {actions[best_action]} ")


La mejor acción seleccionada es: Arriba 
